## Importing required library

In [163]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Gathering required data:

In [164]:
df_rating=pd.read_csv(r"C:\MachineLearning\dataset\ratings.csv")
df_movies=pd.read_csv(r"C:\MachineLearning\dataset\movies.csv")

## Cleaning Data: 

In [165]:
df_rating=df_rating.dropna()

In [166]:
df_rating=df_rating[df_rating.rating!=0]

In [167]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22884377 entries, 0 to 22884376
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 873.0 MB


In [168]:
df_rating=df_rating.drop("timestamp",axis=1)

In [169]:
df_rating.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


## Main User and his movies:

In [170]:
sample=df_rating.sample(1)

In [171]:
user=sample.userId.values[0]
user

189116

In [172]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()

In [173]:
df_user=df_rating[df_rating.userId.isin([user])].sort_values("rating",ascending=False)
#df_user.rating=mms.fit_transform(df_user.rating.values.reshape(-1,1))
main_user,test=train_test_split(df_user,test_size=0.2,random_state=2,shuffle=True)
main_user_ratings=main_user
main_user,'\t',len(test)

(          userId  movieId  rating
 17485040  189116        6     4.0
 17485050  189116      104     4.0
 17485076  189116      842     3.0
 17485042  189116       32     3.0
 17485067  189116      784     3.0
 17485063  189116      743     2.0
 17485041  189116        9     3.0
 17485065  189116      780     3.0
 17485072  189116      832     4.0
 17485085  189116     1459     3.0
 17485046  189116       63     3.0
 17485054  189116      494     3.0
 17485077  189116      849     3.0
 17485051  189116      141     3.0
 17485058  189116      662     4.0
 17485068  189116      785     4.0
 17485045  189116       62     4.0
 17485084  189116     1407     4.0
 17485056  189116      637     3.0
 17485082  189116     1393     4.0
 17485044  189116       61     3.0
 17485064  189116      765     3.0
 17485078  189116      879     3.0
 17485086  189116     1466     3.0
 17485053  189116      376     3.0
 17485079  189116      880     3.0
 17485073  189116      833     2.0
 17485043  189116   

## Neighbours who has watched users movies:

In [176]:
#taking 20000 neighbours movies:
neighbors_all_movies=df_rating[df_rating.movieId.isin(main_user.movieId)].query("userId!='{}'".format(user)).query("rating>4").sort_values("userId")
#neighbors_all_movies.rating=mms.transform(neighbors_all_movies.rating.values.reshape(-1,1))
neighbors_all_movies=df_rating[df_rating.userId.isin(neighbors_all_movies.userId.unique()[0:1000])]

In [177]:
#connecting neighbors and main user
pivot_data=pd.concat((neighbors_all_movies,main_user),axis=0)
pivot_data

,userId,movieId,rating
749,15,1,4.0
750,15,7,3.0
751,15,17,3.5
752,15,24,4.0
753,15,32,4.5
...,...,...,...
17485080,189116,891,3.0
17485061,189116,733,4.0
17485075,189116,839,1.0
17485071,189116,805,3.0


## rearraging data for correlation calculation:

In [179]:
#user_movie_rating table
user_movie_matrix=pivot_data.pivot(columns='movieId',index='userId',values='rating').fillna(0)
user_matrix=user_movie_matrix.copy(deep=True)
user_movie_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,148626,148652,148671,148775,149354,149650,149739,150548,150776,150780
userId,,,,,,,,,,,,,,,,,,,,,
15,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,5.0,3.0,2.0,3.0,4.0,0.0,3.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3826,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
#ymean:
ymean=user_movie_matrix.mean(axis=1)
ymean

userId
15        0.139673
17        0.250750
31        0.024316
37        0.105869
38        0.099382
            ...   
3821      0.065137
3826      0.011474
3828      0.008650
3829      0.041041
189116    0.010503
Length: 1001, dtype: float64

In [181]:
#xi
columns=user_movie_matrix.loc[user].index
main_user_ratings=pd.DataFrame(user_movie_matrix.loc[user].values.reshape(1,-1),columns=columns)
main_user_ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,148626,148652,148671,148775,149354,149650,149739,150548,150776,150780
0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [182]:
#xmean:
x=np.array(main_user_ratings).mean(axis=1)
xmean=x[0]
x[0]

0.010503089143865843

In [183]:
#xi-xmean
xi_xmean=main_user_ratings
for i in xi_xmean.columns:
    xi_xmean[i]=xi_xmean[i]-xmean
xi_xmean

movieId,1,2,3,4,5,6,7,8,9,10,...,148626,148652,148671,148775,149354,149650,149739,150548,150776,150780
0,-0.010503,-0.010503,-0.010503,-0.010503,-0.010503,3.989497,-0.010503,-0.010503,2.989497,-0.010503,...,-0.010503,-0.010503,-0.010503,-0.010503,-0.010503,-0.010503,-0.010503,-0.010503,-0.010503,-0.010503


In [184]:
#yi-ymean
ymean_matrix=user_movie_matrix
for i in ymean_matrix.index:
    ymean_matrix.loc[i,:]-=ymean

In [185]:
yi_ymean=ymean_matrix.fillna(0)

In [186]:
ymean_matrix.to_csv("C:\MachineLearning\dataset\yi_ymean.csv")

In [187]:
#(xi_xmean)(yi_mean)
yi_ymean

movieId,1,2,3,4,5,6,7,8,9,10,...,148626,148652,148671,148775,149354,149650,149739,150548,150776,150780
userId,,,,,,,,,,,,,,,,,,,,,
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [188]:
#(xi-xmean)^2,(yi-ymean)^2
xi_xmean2=(xi_xmean)**2
yi_ymean2=(yi_ymean)**2
yi_ymean2

movieId,1,2,3,4,5,6,7,8,9,10,...,148626,148652,148671,148775,149354,149650,149739,150548,150776,150780
userId,,,,,,,,,,,,,,,,,,,,,
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [189]:
#E(xi-xmean)^2,E(yi-ymean)^2
sum_xi_xmean2=xi_xmean2.sum(axis=1)
sum_yi_ymean2=yi_ymean2.sum(axis=1)
sum_yi_ymean2

userId
15         757.577380
17        2161.832411
31          85.169481
37         501.711361
38         371.011184
             ...     
3821       573.998960
3826       165.838060
3828        10.999755
3829       405.075306
189116     102.803082
Length: 1001, dtype: float64

In [190]:
#(E(xi-xmean)^2)^1/2,(E(yi-ymean)^2)^1/2
sqrt_sum_xi_xmean2=(sum_xi_xmean2)**(1/2)
sqrt_sum_yi_ymean2=(sum_yi_ymean2)**(1/2)

In [191]:
sqrt_sum_yi_ymean2,sqrt_sum_xi_xmean2
#for i in sqrt_sum_xi_xmean2:
#    sqrt_sum_xi_xmean2_x=i

(userId
 15        27.524124
 17        46.495510
 31         9.228731
 37        22.398914
 38        19.261651
             ...    
 3821      23.958275
 3826      12.877813
 3828       3.316588
 3829      20.126483
 189116    10.139185
 Length: 1001, dtype: float64,
 0    10.139185
 dtype: float64)

In [192]:
denominator=sqrt_sum_yi_ymean2*sqrt_sum_xi_xmean2.values[0]
denominator

userId
15        279.072194
17        471.426595
31         93.571818
37        227.106746
38        195.297448
             ...    
3821      242.917398
3826      130.570532
3828       33.627499
3829      204.066141
189116    102.803082
Length: 1001, dtype: float64

In [193]:
xi_xmean_yi_ymean=yi_ymean
for i in xi_xmean_yi_ymean.columns:
    xi_xmean_yi_ymean[i]=xi_xmean_yi_ymean[i]*xi_xmean[i].values[0]
xi_xmean_yi_ymean

movieId,1,2,3,4,5,6,7,8,9,10,...,148626,148652,148671,148775,149354,149650,149739,150548,150776,150780
userId,,,,,,,,,,,,,,,,,,,,,
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
xi_xmean.values.reshape(-1,1).shape

(11330, 1)

In [195]:
#E[(xi-xmean)(yi-ymean)]
numerator=xi_xmean_yi_ymean.sum(axis=1)
numerator

userId
15          3.873036
17         -1.856090
31          6.872661
37         20.135305
38         16.781002
             ...    
3821       -0.500265
3826       16.190159
3828        8.659621
3829        4.037842
189116    102.803082
Length: 1001, dtype: float64

In [242]:
#similarity index/weights
coeff_norm=numerator/denominator
coeff_norm

userId
15        0.013878
17       -0.003937
31        0.073448
37        0.088660
38        0.085925
            ...   
3821     -0.002059
3826      0.123996
3828      0.257516
3829      0.019787
189116    1.000000
Length: 1001, dtype: float64

# weighted average of the movie rating from user movie matrix with respect to coefficient
we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

In [243]:
#multiplying movie matrix by its weight
for i in user_matrix.index:
    user_matrix.loc[i,:]=user_matrix.loc[i,:]*coeff_norm[i]
#weighted sum of ratings of the movie
movie_profile=user_matrix.sum(axis=0)
movie_profile

movieId
1         1.386510e-03
2         1.411840e-06
3         1.877146e-03
4         8.529271e-09
5         1.957492e-03
              ...     
149650    1.719377e-14
149739    3.288231e-21
150548    1.127548e-10
150776    6.950708e-23
150780    9.267610e-23
Length: 11330, dtype: float64

In [244]:
#weighted average of the movies
for i in movie_profile.index:
    movie_profile[i]=(movie_profile[i]/(coeff_norm[pivot_data[pivot_data.movieId==i].userId].sum()))
movie_profile.sort_values(ascending=False)

movieId
879     1.113213e+00
662     6.597011e-01
782     6.517719e-01
891     5.830138e-01
1459    4.774575e-01
            ...     
1237   -9.120562e-10
2946   -1.353660e-09
1279   -2.365941e-09
3079   -3.417925e-08
1264   -2.678489e-07
Length: 11330, dtype: float64

In [252]:
#fetching highly recommended movies
movie=movie_profile.sort_values(ascending=False)
movie=movie[~movie.isin(main_user.movieId)]
movie=movie[movie>0]
movie

movieId
879     1.113213e+00
662     6.597011e-01
782     6.517719e-01
891     5.830138e-01
1459    4.774575e-01
            ...     
4853    3.400634e-22
3628    3.400634e-22
3279    2.579391e-22
5492    2.267090e-22
6152    2.267090e-22
Length: 8045, dtype: float64

In [253]:
#testing with the main users test set
l=0
print(len(test))
for i in movie.index:
    for j in test.movieId:
        if i==j:
            l+=1
print("out of {} movies our viewer has viewed {} movies that were recommended(already watched)".format(len(test),l))

10
out of 10 movies our viewer has viewed 10 movies were recommended(already watched)
